# MONAI Deploy pipeline

## Install Dependencies

In [1]:
!pip install --upgrade monai-deploy-app-sdk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 162 kB 664 kB/s eta 0:00:01


# Import Dependencies
Here, we need to import monai deploy classes, as well as the transforms that we used from MONAI.

In [1]:
import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import (
    AddChannel,
    Compose,
    Lambda,
    LoadImage,
    Resize,
    ScaleIntensity,
)

PNEUMONIA_CLASSES = ["NORMAL", "PNEUMONIA"]

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Creating our operator
Here we define our operator as follows:
1. First we define an input of type DataPath from DISK.
2. Next, we define an output of type DataPath from DISK.
3. Third, we ensure that monai is a dependency that needs to be installed.
4. Then, we define the class, inheriting the Operator class from MONAI Deploy
5. Then we add a property called transform which will do the validation style transforms that we defined in the training module.
6. We then write the compute function, which should consist of getting the path from the input context, using transform on the path, converting the output to an output tensor that is then put on the GPU, and lastly the model is used to perform a forward pass with the image tensor.
7. Lastly, the results are postprocessed into the classes using argmax and the results are saved to output.json.

In [2]:
@md.input("image", DataPath, IOType.DISK)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class PneumoniaClassifierOperator(Operator):
    """Classifies the given image and returns the class name."""

    @property
    def transform(self):
        val_transforms = Compose(
            [
                LoadImage(image_only=True),
                Lambda(func=lambda x: np.mean(x, axis=2) if len(x.shape) >= 3 else x),
                AddChannel(),
                ScaleIntensity(),
                Resize(spatial_size=(224,224)),
            ]
        )
        return val_transforms

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch
        
        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        image_tensor = self.transform(input_path)  # (1, 224, 224), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 224, 224), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # get a TorchScriptModel object

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = PNEUMONIA_CLASSES[output_classes[0]]  # get the class name
        print(result)

        # Get output (folder) path and create the folder if not exists
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # Write result to "output.json"
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)

# Creating our application
In this section, all the operators that are defined should be included. Here, we only used one operator, so we will use the add_operator function to add our operator into the application function. Notice here that you can also define the number of cpu, gpu and memory that are required.

In [3]:
@md.resource(cpu=1, gpu=1, memory="1Gi")
class App(Application):
    """Application class for the Pneumonia classifier."""

    def compose(self):
        classifier_op = PneumoniaClassifierOperator()

        self.add_operator(classifier_op)

# Testing our application
In order to test our application, we will point "test_input_path" to a jpeg file in our test folder.

In [4]:
test_input_path = "./chest_xray/test/NORMAL/IM-0001-0001.jpeg"
print(f"Test input file path: {test_input_path}")

Test input file path: ./chest_xray/test/NORMAL/IM-0001-0001.jpeg


Next, we instantiate the App class, and then perform run.

In [5]:
app = App()

In [6]:
app.run(input=test_input_path, output="output", model="classifier.zip")

Going to initiate execution of operator PneumoniaClassifierOperator
Executing operator PneumoniaClassifierOperator (Process ID: 2057, Operator ID: 84a872ef-a8f1-4230-898c-843d25d11408)
NORMAL
Done performing execution of operator PneumoniaClassifierOperator



Lastly, we can check that the results were indeed output into a json file.

In [7]:
!cat output/output.json

"NORMAL"

# Wrapping it all up
Next, we take cells 1, 2, and 3, and paste them into a file called pneumonia_classifier_monaideploy.py. This is done in the next cell.
At the end, we add the `if __name__ == "__main__":` definition to make sure that the app is run if it's called on the python command line.

In [8]:
%%writefile pneumonia_classifier_monaideploy.py

# Cell [1]
import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)
from monai.transforms import (
    AddChannel,
    Compose,
    Lambda,
    LoadImage,
    Resize,
    ScaleIntensity,
)

PNEUMONIA_CLASSES = ["NORMAL", "PNEUMONIA"]

# Cell [2]
@md.input("image", DataPath, IOType.DISK)
@md.output("output", DataPath, IOType.DISK)
@md.env(pip_packages=["monai"])
class PneumoniaClassifierOperator(Operator):
    """Classifies the given image and returns the class name."""

    @property
    def transform(self):
        val_transforms = Compose(
            [
                LoadImage(image_only=True),
                Lambda(func=lambda x: np.mean(x, axis=2) if len(x.shape) >= 3 else x),
                AddChannel(),
                ScaleIntensity(),
                Resize(spatial_size=(224,224)),
            ]
        )
        return val_transforms

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        import json

        import torch
        
        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        image_tensor = self.transform(input_path)  # (1, 224, 224), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 224, 224), torch.float32

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        image_tensor = image_tensor.to(device)

        model = context.models.get()  # get a TorchScriptModel object

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = PNEUMONIA_CLASSES[output_classes[0]]  # get the class name
        print(result)

        # Get output (folder) path and create the folder if not exists
        output_folder = op_output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # Write result to "output.json"
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)
            
# Cell [3]
@md.resource(cpu=1, gpu=1, memory="1Gi")
class App(Application):
    """Application class for the Pneumonia classifier."""

    def compose(self):
        classifier_op = PneumoniaClassifierOperator()

        self.add_operator(classifier_op)

# Finally
if __name__ == "__main__":
    App(do_run=True)

Writing pneumonia_classifier_monaideploy.py


# Use PYTHON to run the code directly
Now that the code is written, we can then run the code using python directly as a test.

In [10]:
!python pneumonia_classifier_monaideploy.py -i {test_input_path} -o output -m classifier.zip

Going to initiate execution of operator PneumoniaClassifierOperator
Executing operator PneumoniaClassifierOperator (Process ID: 2110, Operator ID: 59b054f0-7729-4910-bb43-69faf2aa8480)
NORMAL
Done performing execution of operator PneumoniaClassifierOperator



# Use MONAI Deploy Runner to test the app
We can also test the execution using the included MONAI deploy runner.

In [11]:
!monai-deploy exec pneumonia_classifier_monaideploy.py -i {test_input_path} -o output -m classifier.zip

Going to initiate execution of operator PneumoniaClassifierOperator
Executing operator PneumoniaClassifierOperator (Process ID: 2141, Operator ID: 2e08c7c1-2b10-44b1-b816-7621ba5c765f)
NORMAL
Done performing execution of operator PneumoniaClassifierOperator



# Package App for Docker
Lastly, this step will call docker to package the application as a docker image which can then be run anywhere that docker images are supported.

In [12]:
!monai-deploy package pneumonia_classifier_monaideploy.py --tag pneumonia_app:latest --model classifier.zip  # -l DEBUG

Building MONAI Application Package... /bin/bash: docker: command not found
Done
